<a href="https://colab.research.google.com/github/amalsalilan/Infosys-Springboard-Virtual-Internship-6.0-Open-Deep-Researcher-batch-2/blob/Prateek_Ray/calrifybot_researchberif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip uninstall -y google-generativeai google-ai-generativelanguage
!pip install google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15
!pip install --upgrade langchain-google-genai langchain

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: google-ai-generativelanguage 0.7.0
Uninstalling google-ai-generativelanguage-0.7.0:
  Successfully uninstalled google-ai-generativelanguage-0.7.0
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
Using cached google_generativeai-0.8.5-py3-none-any.whl (155 kB)
Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl (1.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.12 requires google-ai-generativelanguage<1,>=0.7, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


  Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl.metadata (10 kB)
Using cached google_ai_generativelanguage-0.7.0-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [1]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 5.5 MB/s eta 0:00:00


In [1]:
import os

os.environ["GOOGLE_API_KEY"] = ("YOUR_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = ("YOUR_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "clarify-chatbot"

# Tavily (transport/search) API key — optional; if missing we will simulate results
os.environ["TAVILY_API_KEY"] = ("YOUR_API_KEY")

MODEL_NAME = "gemini-2.5-pro"


In [2]:
from typing import TypedDict, Optional, Dict, List
import time, json, re, math, textwrap, datetime, requests, os

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

from langgraph.graph import StateGraph, END

# Initialize LLM (keep deterministic for reproducibility)
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)

# Helper for safe text extraction
def invoke_text(result):
    if isinstance(result, dict):
        return result.get("text") or result.get("content") or str(result)
    return str(result)


In [18]:
class ClarifyState(TypedDict):
    query: str
    missing_info: Optional[str]
    clarification: Optional[str]
    answer: Optional[str]
    memory: Dict[str, str]

# Analysis prompt
analysis_prompt = PromptTemplate(
    input_variables=["query", "memory_str"],
    template="""
User memory:
{memory_str}

Check if the user query needs a short clarification. If OK to answer, just say "OK".
If more info is needed, reply exactly like this:
CLARIFY: <what you need in 1 sentence>

User query:
{query}
"""
)
analysis_chain = LLMChain(llm=llm, prompt=analysis_prompt)

def analyze_node(state: ClarifyState):
    memory_str = "\n".join([f"{k}: {v}" for k,v in state.get("memory", {}).items()])
    raw = analysis_chain.invoke({"query": state.get("query",""), "memory_str": memory_str})
    raw_text = invoke_text(raw).strip()
    if raw_text.upper().startswith("CLARIFY:"):
        state["missing_info"] = raw_text.split("CLARIFY:",1)[1].strip()
    else:
        state["missing_info"] = None
    return state

def clarify_node(state: ClarifyState):
    missing = state.get("missing_info")
    if missing:
        print("Bot:", f"Can you clarify: {missing}?")
        user_ans = input("You (clarification): ").strip()
        if user_ans.lower() in ("exit","quit"):
            print("Goodbye!")
            raise SystemExit()
        state["clarification"] = user_ans
        # quick memory extraction heuristics
        lower = user_ans.lower()
        if "my name is" in lower:
            name = re.split(r"my name is", lower, 1)[1].strip().split()[0].title()
            state["memory"]["name"] = name
        elif "i live in" in lower or "my city is" in lower:
            city = re.sub(r"(i live in|my city is)", "", lower).strip().title()
            state["memory"]["city"] = city
        else:
            key = missing.replace(" ", "_").lower()
            state["memory"][key] = user_ans
    return state


In [19]:
def update_memory_from_text(text: str, memory: dict) -> dict:
    lower = text.lower()
    new_memory = memory.copy()
    m = re.search(r"\bmy name is (\w+)", lower)
    if m: new_memory["name"] = m.group(1).title()
    m = re.search(r"\bi live in ([a-z\s]+)", lower)
    if m: new_memory["city"] = m.group(1).title()
    m = re.search(r"\bfrom ([a-z\s]+) to ([a-z\s]+)", lower)
    if m:
        new_memory["origin"] = m.group(1).title()
        new_memory["destination"] = m.group(2).title()
    m = re.search(r"(\d+[,\d]*)\s*(inr|usd|rs|rupees|₹)", lower)
    if m: new_memory["budget"] = m.group(1)
    m = re.search(r"\b(\d{1,2}\s+\w+\s*\d{4}?)", text)
    if m: new_memory["date"] = m.group(1)
    return new_memory




In [33]:
TAVILY_KEY = os.environ.get("TAVILY_API_KEY") or ""

def tavily_search(query: str, location: Optional[str]=None, budget: Optional[str]=None, limit=5):
    """
    Query Tavily if key present, otherwise return simulated results.
    Expected return: list of dicts with keys: title, url, snippet, price
    """
    if not TAVILY_KEY:
        # Simulated results for dev when API not available
        simulated = [
            {"title":"Sample Flight - Airline A", "url":"https://example.com/flightA", "snippet":"one-stop via place", "price":"₹80,000"},
            {"title":"Sample Hotel - Budget Inn", "url":"https://example.com/hotel1", "snippet":"3-star near Strip", "price":"₹6,000/night"},
            {"title":"Sample Train/Bus Option", "url":"https://example.com/train", "snippet":"connect via hub", "price":"₹5,000"}
        ]
        return simulated[:limit]
    # Real Tavily call — adapt to Tavily API docs
    headers = {"Authorization": f"Bearer {TAVILY_KEY}", "Content-Type":"application/json"}
    payload = {"q": query, "location": location, "budget": budget, "limit": limit}
    try:
        resp = requests.post("https://api.tavily.com/v1/search", headers=headers, json=payload, timeout=15)
        resp.raise_for_status()
        data = resp.json()
        # map to expected shape (depends on API output)
        results = []
        for item in data.get("results", [])[:limit]:
            results.append({
                "title": item.get("title"),
                "url": item.get("url"),
                "snippet": item.get("snippet") or item.get("description", ""),
                "price": item.get("price")
            })
        return results
    except Exception as e:
        print("Tavily search failed:", e)
        # fallback to empty list
        return []

def run_deep_research(query: str, origin: Optional[str]=None, budget: Optional[str]=None):
    # 1) gather raw results
    flights = tavily_search(f"{query} cheapest flights {origin or ''}", limit=5)
    hotels = tavily_search(f"{query} hotels stays {origin or ''}", limit=5)
    activities = tavily_search(f"{query} things to do {origin or ''}", limit=5)

    # Add static URLs for top flight aggregators and hotels
    flight_links = [
        {"title":"Skyscanner", "url":"https://www.skyscanner.co.in"},
        {"title":"Kayak", "url":"https://www.kayak.co.in"},
        {"title":"Google Flights", "url":"https://www.google.com/flights"}
    ]
    hotel_links = [
        {"title":"Booking.com", "url":"https://www.booking.com"},
        {"title":"Agoda", "url":"https://www.agoda.com"},
        {"title":"Rove Hotels", "url":"https://www.rovehotels.com"}
    ]

    # Merge static links into flights/hotels JSON so LLM sees them
    flights_json = json.dumps(flights + flight_links, ensure_ascii=False)
    hotels_json = json.dumps(hotels + hotel_links, ensure_ascii=False)
    activities_json = json.dumps(activities, ensure_ascii=False)
    today = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")

    # LLM prompt
    synth_prompt = PromptTemplate(
        input_variables=["query","origin","budget","flights","hotels","activities","today"],
        template="""
You are an expert travel researcher. Using the raw results below, produce a travel research brief.
Include:
1) One-paragraph executive summary.
2) A compact table of the top 3 transport options with cost and boarding/offboarding details and source URL (CSV-style).
3) Top 3 hotel/stay picks (name, nightly, area, booking URL).
4) 5 bullet-point practical tips (visa, transit time, cheapest booking tips).
5) List of source URLs.

Flights raw:
{flights}

Hotels raw:
{hotels}

Activities raw:
{activities}

Origin: {origin}
Budget: {budget}
Query: {query}
Date produced: {today}
"""
    )

    synth_chain = LLMChain(llm=llm, prompt=synth_prompt)
    prompt_vars = {
        "query": query,
        "origin": origin or "",
        "budget": budget or "",
        "flights": flights_json,
        "hotels": hotels_json,
        "activities": activities_json,
        "today": today
    }
    response = synth_chain.invoke(prompt_vars)
    brief_text = invoke_text(response)

    # create simple table structure from the top items (best-effort)
    table = []
    sources = [f["url"] for f in flight_links + hotel_links]  # start with static URLs
    for s in (flights[:3] + hotels[:3] + activities[:3]):
        url = s.get("url")
        if url: sources.append(url)

    return {"brief": brief_text, "table": table, "sources": list(dict.fromkeys(sources))}


In [23]:
def stream_text_from_llm_chain(chain: LLMChain, inputs: dict, chunk_size=80, delay=0.02):
    try:
        if hasattr(chain, "stream") and callable(chain.stream):
            tokens = []
            for token_chunk in chain.stream(inputs):
                t = token_chunk.get("text") if isinstance(token_chunk, dict) else str(token_chunk)
                print(t, end="", flush=True)
                tokens.append(t)
            print()
            return "".join(tokens)
    except Exception:
        pass
    res = chain.invoke(inputs)
    text = invoke_text(res)
    for i in range(0, len(text), chunk_size):
        print(text[i:i+chunk_size], end="", flush=True)
        time.sleep(delay)
    print()
    return text



In [24]:
def answer_node(state: ClarifyState):
    clarification = state.get("clarification") or ""
    memory_str = "\n".join([f"{k}: {v}" for k,v in state.get("memory", {}).items()])

    # Determine whether deep research is required
    run_text = stream_text_from_llm_chain(answer_chain, {
        "query": state["query"],
        "clarification": clarification,
        "memory_str": memory_str
    }, chunk_size=120, delay=0.01)

    run_text_stripped = (run_text or "").strip()

    if run_text_stripped == "DEEP_RESEARCH_NEEDED":
        # parse origin & budget from memory or query/clarification
        origin = state["memory"].get("city") or state["memory"].get("origin") or None
        budget = None
        m = re.search(r"(\d+[,\d]*)\s*(inr|usd|rs|rupees|₹)?", state["query"].lower())
        if m:
            budget = m.group(1)

        # run deep research
        research = run_deep_research(state["query"], origin=origin, budget=budget)
        brief_text = research.get("brief", "No research data available.")
        state["answer"] = brief_text

        # stream the brief token-by-token
        print("\n--- Research Brief ---")
        for i in range(0, len(brief_text), 120):
            print(brief_text[i:i+120], end="", flush=True)
            time.sleep(0.02)
        print("\n--- Sources ---")
        for s in research.get("sources", []):
            print("-", s)
    else:
        state["answer"] = run_text or "No answer generated."

    # quick memory extraction from answer
    update_memory_from_text(state["answer"] or "", state["memory"])
    print()  # newline
    return state



In [25]:
def reflection_node(state: ClarifyState):
    """
    Evaluates the answer for completeness, accuracy, and relevance.
    Assigns a quality score (0-100) and improvement suggestions.
    """
    content = state.get("answer") or "No content to reflect on."

    reflection_prompt = PromptTemplate(
        input_variables=["content"],
        template="""
You are a travel content evaluator. Evaluate the provided research report for:
1) Completeness (flights, hotels, activities)
2) Accuracy (budget, dates)
3) Usefulness and clarity for the user

Score the content 0-100.
If below 80, provide brief suggestions to improve.

Content:
{content}
"""
    )
    reflection_chain = LLMChain(llm=llm, prompt=reflection_prompt)

    reflection_text = stream_text_from_llm_chain(reflection_chain, {"content": content}, chunk_size=80, delay=0.01)
    reflection_text = reflection_text.strip()

    # Extract score and suggestions (simple parsing)
    score_match = re.search(r"(\d{1,3})", reflection_text)
    score = int(score_match.group(1)) if score_match else 1
    status = "OK" if score >= 80 else "IMPROVE"

    # Extract improvement suggestions if score < 80
    suggestions_match = re.search(r"(?:suggestions?:\s*)(.+)", reflection_text, re.IGNORECASE | re.DOTALL)
    suggestions = suggestions_match.group(1).strip() if suggestions_match else ""

    state["reflection_score"] = score
    state["quality_score"] = score
    state["reflection_status"] = status
    state["improvement_suggestions"] = suggestions

    print(f"\nReflection result: Score = {score}/100, Status = {status}")
    if suggestions:
        print("Improvement suggestions:", suggestions)
    return state




In [28]:
workflow = StateGraph(ClarifyState)
workflow.add_node("analyze", analyze_node)
workflow.add_node("clarify", clarify_node)
workflow.add_node("answer", answer_node)
workflow.set_entry_point("analyze")
workflow.add_conditional_edges("analyze", lambda s: "clarify" if s.get("missing_info") else "answer",
                               {"clarify":"clarify","answer":"answer"})
workflow.add_edge("clarify","answer")
workflow.add_edge("answer", END)
app = workflow.compile()

def clarify_with_user_memory():
    print("Type 'exit' to quit.\n")
    memory_dict: Dict[str,str] = {}
    followups: List[str] = []

    while True:
        # choose input prompt depending on whether followups shown previously
        if followups:
            user_input = input("You (or choose 1/2/3): ").strip()
            # if user typed a digit, select followup
            if user_input.isdigit():
                idx = int(user_input)
                if 1 <= idx <= len(followups):
                    user_input = followups[idx-1]
                    print(f"You selected follow-up: {user_input}")
        else:
            user_input = input("You: ").strip()

        if user_input.lower() in ("exit","quit"):
            print("Goodbye!")
            break

        # update memory heuristics immediately with user-provided facts
        memory_dict = update_memory_from_text(user_input, memory_dict)

        # build state
        state: ClarifyState = {
            "query": user_input,
            "missing_info": None,
            "clarification": "",
            "answer": None,
            "memory": memory_dict
        }

        # run graph
        final_state = app.invoke(state)
        if isinstance(final_state, dict):
            final_state = final_state

        # safe memory update
        memory_dict.update(final_state.get("memory", {}))
        memory_dict = update_memory_from_text(final_state.get("answer","") or "", memory_dict)

        # reflection-based regeneration loop
        # only regenerate if reflection_status == "IMPROVE" AND score < 90
        while final_state.get("reflection_status") == "IMPROVE" and (final_state.get("quality_score") or 0) < 90:
            print("\nRegenerating content based on reflection suggestions...")
            improvement_note = final_state.get("improvement_suggestions") or ""
            state["clarification"] = ((state.get("clarification") or "") + " " + improvement_note).strip()
            final_state = answer_node(state)
            final_state = reflection_node(final_state)
            # update memory again after regeneration
            memory_dict.update(final_state.get("memory", {}))
            memory_dict = update_memory_from_text(final_state.get("answer","") or "", memory_dict)

        # show followups
        followups = suggest_followups(final_state.get("answer","") or "")
        if followups:
            print("\nPossible follow-ups:")
            for i,s in enumerate(followups,1):
                print(f"{i}. {s}")

        # display reflection and quality score
        print(f"\nReflection result: Score = {final_state.get('quality_score',0)}/100, Status = {final_state.get('reflection_status','UNKNOWN')}")
        print("\n=== Final Research Report / Answer ===")
        print(final_state.get("answer","None"))
        print(f"\nQuality Score: {final_state.get('quality_score',0)}/100")

        # debugging memory
        print("Current memory:", memory_dict)
        print("-"*40)


In [34]:
clarify_with_user_memory()


Type 'exit' to quit.

You: make a round trip plan of 1 week from delhi to dubai on 01-11-2025. it will be a solo trip and budget will 1 lakh and suggest cheap flights and stays
DEEP_RESEARCH_NEEDED
Tavily search failed: 404 Client Error: Not Found for url: https://api.tavily.com/v1/search
Tavily search failed: 404 Client Error: Not Found for url: https://api.tavily.com/v1/search
Tavily search failed: 404 Client Error: Not Found for url: https://api.tavily.com/v1/search


/tmp/ipython-input-3145850897.py:60: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")



--- Research Brief ---
Of course. Here is the travel research brief based on your request.

***

### **Travel Research Brief: Delhi to Dubai**

**1. Executive Summary**

This brief outlines a one-week solo trip from Delhi to Dubai from November 1-8, 2025, with a total budget of ₹1,00,000. The plan is highly feasible, focusing on cost-effective travel and accommodation. The core recommendation is to book flights on budget carriers like IndiGo or SpiceJet approximately 4-6 months in advance for optimal pricing, estimated at ₹20,000-₹25,000. For accommodation, staying in well-connected, budget-friendly areas like Deira or Bur Dubai is advised, with recommended hotels averaging ₹5,500-₹7,000 per night. This strategy leaves a substantial portion of the budget for activities, food, and local transport, ensuring a comfortable and comprehensive travel experience.

**2. Top 3 Transport Options**

| Option | Est. Round-Trip Cost (INR) | Boarding/Offboarding | Source |
| :--- | :--- | :--- | :--

* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 14.532861571s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 14
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 12.490290246s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }



Possible follow-ups:
1. Based on the answer provided, here are 3 short follow-up questions:
2. Can you suggest a 7-day itinerary that balances major attractions with budget-friendly activities?
3. How would you break down the remaining budget for daily expenses like food, local transport, and attraction tickets?

Reflection result: Score = 0/100, Status = UNKNOWN

=== Final Research Report / Answer ===
Of course. Here is the travel research brief based on your request.

***

### **Travel Research Brief: Delhi to Dubai**

**1. Executive Summary**

This brief outlines a one-week solo trip from Delhi to Dubai from November 1-8, 2025, with a total budget of ₹1,00,000. The plan is highly feasible, focusing on cost-effective travel and accommodation. The core recommendation is to book flights on budget carriers like IndiGo or SpiceJet approximately 4-6 months in advance for optimal pricing, estimated at ₹20,000-₹25,000. For accommodation, staying in well-connected, budget-friendly areas lik